In [ ]:
from Bio import SeqIO

import os
from os import path
from os.path import isfile, join, dirname, isdir, exists


import fnmatch
import gffpandas.gffpandas as gffpd

import matplotlib.pyplot as plt
import numpy as np

def make_dir(*argv):
    mydir = path.join(*argv)    
    if not path.exists(mydir):        
        if len(argv) > 1:
            make_dir(*argv[:-1])            
        os.mkdir(mydir)
    return mydir


def make_path(*argv):
    mypath = path.join(*argv)
    if not path.exists(dirname(mypath)):
        make_dir(*argv[:-1])
    return mypath

In [ ]:
#tool for searching gff table and finding a row given a gene id

ecotype = '10015'
gene_id = 'T029-R1'

annotation = gffpd.read_gff3('gff/%s.gff'%ecotype)

df = annotation.df

foundGene = False
exon_start_l = []
exon_end_l = []

for ii in range(df.shape[0]):
    if df.iloc[ii]['type'] == 'mRNA':
        if not foundGene:
            if df.iloc[ii]['attributes'].split('|')[1].split(';')[0] == gene_id:
                foundGene = True
                startGene = df.iloc[ii]['start']
                print(ii)
        else:
            break
    elif df.iloc[ii]['type'] == 'exon':
        if foundGene:
            exon_start_l.append(df.iloc[ii]['start'])
            exon_end_l.append(df.iloc[ii]['end'])

exon_start_l = [x - startGene for x in exon_start_l]
exon_end_l = [x - startGene for x in exon_end_l] 

In [ ]:
ecotype = '10015'
annotation = gffpd.read_gff3(make_path('gff', ecotype+'.gff'))

In [ ]:
print(annotation.df)

In [ ]:
df = annotation.df

In [ ]:
df.head(30)

In [ ]:
#make a list of gene id's

#list comprehension
gene_l = [attr.split('|')[1].split(';')[0] for attr in df['attributes'][df['type']=='mRNA']]

#for loop
if False:
    gene_l = []
    for attr in df['attributes'][df['type']=='mRNA']:
        gene_l.append(attr.split('|')[1].split(';')[0])
        
#check if there are no redundancies in gene list
len(gene_l) == len(set(gene_l))

In [ ]:
#A more efficient way of searching the table. Doesn't find non-contiguous annotations, though.

if False:
    gene_id = 'T379-R1'

    foundGene = False
    exon_start_l = []
    exon_end_l = []

    for ii in range(df.shape[0]):
        if df.iloc[ii]['type'] == 'mRNA':
            if not foundGene:
                if df.iloc[ii]['attributes'].split('|')[1].split(';')[0] == gene_id:
                    foundGene = True
                    startGene = df.iloc[ii]['start']
            else:
                break
        elif df.iloc[ii]['type'] == 'exon':
            if foundGene:
                exon_start_l.append(df.iloc[ii]['start'])
                exon_end_l.append(df.iloc[ii]['end'])
                
    exon_start_l = [x - startGene for x in exon_start_l]
    exon_end_l = [x - startGene for x in exon_end_l]            

    plt.figure(figsize = (10, 3))

    for xmin, xmax in zip(exon_start_l, exon_end_l):
        plt.hlines(y='', xmin=xmin, xmax=xmax, color='b')                

In [ ]:
#A more efficient way of searching the table. Doesn't find non-contiguous annotations, though.

if True:
    gene_id = 'T319-R1'

    foundGene = False
    exon_start_l = []
    exon_end_l = []

    for ii in range(df.shape[0]):
        if df.iloc[ii]['type'] == 'mRNA':
            if not foundGene:
                if df.iloc[ii]['attributes'].split('|')[1].split(';')[0] == gene_id:
                    foundGene = True
                    startGene = df.iloc[ii]['start']
                    print(ii)
            else:
                break
        elif df.iloc[ii]['type'] == 'exon':
            if foundGene:
                exon_start_l.append(df.iloc[ii]['start'])
                exon_end_l.append(df.iloc[ii]['end'])
                
    exon_start_l = [x - startGene for x in exon_start_l]
    exon_end_l = [x - startGene for x in exon_end_l]                          

In [ ]:
df[476:].head(10)

In [ ]:
#A more efficient way of searching the table. Doesn't find non-contiguous annotations, though.

if False:
    gene_id = 'T379-R1'

    foundGene = False
    exon_start_l = []
    exon_end_l = []

    for ii in range(df.shape[0]):
        if df.iloc[ii]['type'] == 'mRNA':
            if not foundGene:
                if df.iloc[ii]['attributes'].split('|')[1].split(';')[0] == gene_id:
                    foundGene = True
                    startGene = df.iloc[ii]['start']
            else:
                break
        elif df.iloc[ii]['type'] == 'exon':
            if foundGene:
                exon_start_l.append(df.iloc[ii]['start'])
                exon_end_l.append(df.iloc[ii]['end'])
                
    exon_start_l = [x - startGene for x in exon_start_l]
    exon_end_l = [x - startGene for x in exon_end_l]            

    plt.figure(figsize = (10, 3))

    for xmin, xmax in zip(exon_start_l, exon_end_l):
        plt.hlines(y='', xmin=xmin, xmax=xmax, color='b')  

In [ ]:
#Searching the table for exons in a gene. Finds non-contiguous annotations.

plt.ioff()

for gene_id in gene_l:

    exon_coord_l = []

    for ii in range(df.shape[0]):
        if df.iloc[ii]['type'] == 'exon':
            if df.iloc[ii]['attributes'].split('|')[1].split(';')[0].startswith(gene_id):
                exon_coord_l.append((df.iloc[ii]['start'], df.iloc[ii]['end']))     

    geneStart = min([min(coord) for coord in exon_coord_l])
    exon_coord_zerostart = [(start-geneStart, end-geneStart) for start,end in exon_coord_l]

    fig = plt.figure(figsize = (10, 1))

    for xmin, xmax in exon_coord_zerostart:
        plt.hlines(y='', xmin=xmin, xmax=xmax, color='b')
    fig.tight_layout()        
    fig.patch.set_facecolor('xkcd:white') # make transparent background white
    fig.savefig(make_path('intron_plots', ecotype, gene_id+'.png'), bbox_inches="tight")
    plt.close(fig)

In [ ]:
gene_l

In [ ]:
#A more efficient way of searching the table. Doesn't find non-contiguous annotations, though.

gene_id = 'T378-R1'

foundGene = False
exon_coord_l = []

for ii in range(df.shape[0]):
    if df.iloc[ii]['type'] == 'mRNA':
        if not foundGene:
            if df.iloc[ii]['attributes'].split('|')[1].split(';')[0] == gene_id:
                foundGene = True
        else:
            break
    elif df.iloc[ii]['type'] == 'exon':
        if foundGene:
            exon_coord_l.append((df.iloc[ii]['start'], df.iloc[ii]['end']))   


In [ ]:
exon_coord_l

In [ ]:
exon_start_l = []
exon_end_l = []

gene_exon_start_l = []
gene_exon_end_l = []

geneStart = 2707
geneEnd = 6575

for ii in range(df.shape[0]):
    if df.iloc[ii]['seq_id'] == 'tig00000074':
        if df.iloc[ii]['type'] == 'exon':
            exon_start_l.append(df.iloc[ii]['start'])
# print(exon_start_l)
            exon_end_l.append(df.iloc[ii]['end'])
# print(exon_end_l)

for x, y in zip(exon_start_l, exon_end_l):
    if x >= geneStart and x <= geneEnd: 
        gene_exon_start_l.append(x)
    if y >= geneStart and y <= geneEnd: 
        gene_exon_end_l.append(y)

In [ ]:
xs = [1, 100]

plt.figure(figsize = (10, 7))

plt.vlines(x = gene_exon_start_l, ymin = 0, ymax = max(xs),
           colors = 'blue',
           label = 'vline_multiple - full height')

plt.vlines(x = gene_exon_end_l, ymin = 0, ymax = max(xs),
           colors = 'red',
           label = 'vline_multiple - full height')
plt.title("ID=10015|G379;Alias=maker-tig00000074-snap-gene-0.6;Name=10015|G379;NLR=TNL")

In [ ]:
combined_l = []
for jj, end in enumerate(gene_exon_end_l):
    combined_l.append((gene_exon_start_l[jj], end))
# print(combined_l)

xs = [1, 100]

plt.figure(figsize = (10, 7))

plt.vlines(x = combined_l[0], ymin = 0, ymax = max(xs),
           colors = 'red',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[1], ymin = 0, ymax = max(xs),
           colors = 'orange',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[2], ymin = 0, ymax = max(xs),
           colors = 'yellow',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[3], ymin = 0, ymax = max(xs),
           colors = 'green',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[4], ymin = 0, ymax = max(xs),
           colors = 'blue',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[5], ymin = 0, ymax = max(xs),
           colors = 'purple',
           label = 'vline_multiple - full height')

plt.vlines(x = combined_l[6], ymin = 0, ymax = max(xs),
           colors = 'pink',
           label = 'vline_multiple - full height')
plt.title("ID=10015|G379;Alias=maker-tig00000074-snap-gene-0.6;Name=10015|G379;NLR=TNL")